In [1]:
import AdventOfCode

In [2]:
input <- dayLines 23

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as A
import Data.Int

data Nanobot
    = Nanobot
    { position :: (Integer, Integer, Integer)
    , radius :: Integer
    } deriving (Eq, Show)

parseLine = do
    A.string "pos=<"
    x <- A.signed A.decimal
    A.string ","
    y <- A.signed A.decimal
    A.string ","
    z <- A.signed A.decimal
    A.string ">, r="
    r <- A.decimal
    pure $ Nanobot (x,y,z) r

In [4]:
distance (x,y,z) (a,b,c) = abs (x-a) + abs (y-b) + abs (z-c)


nanobots = map (parsed parseLine) input

In [5]:
import Data.List
import Data.Function (on)

Nanobot pos rad = maximumBy (compare `on` radius) nanobots

In [6]:
length $ filter (\(Nanobot p _) -> distance pos p <= rad) nanobots

408

In [7]:
import Data.SBV

In [8]:
sAbs :: SInteger -> SInteger
sAbs a = ite (a .> 0) a (negate a)

sDistance :: (SInteger, SInteger, SInteger) -> (SInteger, SInteger, SInteger) -> SInteger
sDistance (x,y,z) (a,b,c) = sAbs (x-a) + sAbs (y-b) + sAbs (z-c)

sInRange :: (SInteger, SInteger, SInteger) -> Nanobot -> SBool
sInRange c (Nanobot (x,y,z) r) = sDistance c (literal x, literal y, literal z) .<= literal r

countInRange :: (SInteger, SInteger, SInteger) -> [Nanobot] -> SInteger
countInRange coord = sum . map (oneIf . sInRange coord)

In [9]:
optimize Lexicographic $ do
    [x,y,z] <- sIntegers ["x", "y", "z"]
    maximize "goal" $ countInRange (x,y,z) nanobots
    minimize "distance from origin" $ sDistance (x,y,z) (literal 0, literal 0, literal 0)

Optimal model:
  x                    =  22198541 :: Integer
  y                    =  47809208 :: Integer
  z                    =  51159819 :: Integer
  goal                 =       977 :: Integer
  distance from origin = 121167568 :: Integer